In [ ]:
from datetime import datetime
from io import StringIO
import pandas as pd
import requests

# Exposed endpoints:
# https://asx.api.markitdigital.com/asx-research/1.0 and https://cdn-api.markitdigital.com/apiman-gateway/ASX/asx-research/1.0
# https://asx-api.markitqa.com/asx-research/1.0 and https://cdn.qa-api.markitdigital.com/apiman-gateway/ASX/asx-research/1.0
BASE_URL = 'https://cdn-api.markitdigital.com/apiman-gateway/ASX/asx-research/1.0' # use the non-qa cdn endpoint (should be cached)

# Company requests
def find_all_companies():
  response = requests.get(f'{BASE_URL}/companies/directory/file')
  return pd.read_csv(StringIO(response.text), converters={
    'Listing date': lambda x: datetime.strptime(x, '%d/%m/%Y').date(),
    'Market Cap': lambda x: int(x) if x.isdigit() else -1
  })

# Get the companies current financial details  
def find_company_key_statistics_by_sym(sym):
  return requests.get(f'{BASE_URL}/companies/{sym}/key-statistics').json().data

# Get company address, share-registry, directors, secretaries etc. 
def find_company_about_by_sym(sym):
  return requests.get(f'{BASE_URL}/companies/{sym}/about').json().data

# used to get the xidEntity used by announcements
def search(text):
  return requests.get(f'{BASE_URL}/search/predictive', params={'searchText': text}).data


def find_all_announcements(paging={page=0,itemsPerPage=25},filter={}): #summaryCountsDate=YYYY-MM-DD,includeFacets=bool,entityXids=str,priceSensitiveOnly=bool,startDate=YYYY-MM-DD,endDate=YYYY-MM-DD
  return requests.get(f'{BASE_URL}/markets/announcements').data

# # ASX company csv data ("ASX code","Company name","GICs industry group","Listing date","Market Cap")
# with (
#   requests.get(f'{BASE_URL}/companies/directory/file') as res,
#   open(f'../reference/asx/asx_companies.csv', 'wb') as file
# ):
#   file.write(res.content)

,ASX code,Company name,GICs industry group,Listing date,Market Cap
0,10X,EXULTANT MINING LIMITED,Materials,2025-12-11,7812000
1,14D,1414 DEGREES LIMITED,Capital Goods,2018-09-12,11649736
2,1AD,ADALTA LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",2016-08-22,11476345
3,1AE,AURORA ENERGY METALS LIMITED,Materials,2022-05-18,26859561
4,1AI,ALGORAE PHARMACEUTICALS LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",2004-09-01,35463975
...,...,...,...,...,...
1846,ZLD,ZELIRA THERAPEUTICS LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",2003-07-28,8090065
1847,ZMI,ZINC OF IRELAND NL,Materials,2007-09-18,9894182
1848,ZMM,ZIMI LIMITED,Technology Hardware & Equipment,2007-09-10,5293357
1849,ZNC,ZENITH MINERALS LIMITED,Materials,2007-05-29,49821525
